## Actividad Guiada 2 - Programación Dinámica

Estudiante: Efraín Paucar Castillo

In [4]:
#Carga de datos de tarifas del caso del viajero. Para las rutas inexistentes valores muy altos 999 de precios.
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]]

def Precios(TARIFAS):
  #Total de Nodos
  N = len(TARIFAS[0])
  
  #Inicialización de la tabla de precios
  PRECIOS = [ [9999]*N for i in [9999]*N ]
  RUTA = [ [""]*N for i in [""]*N ]
  
  for i in range(N-1):
    for j in range(i+1,N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
      
      for k in range(i,j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
          MIN = PRECIOS[i][k] + TARIFAS[k][j]
          RUTA[i][j] = k
          
      PRECIOS[i][j] = MIN
  return PRECIOS, RUTA

#Probando los resultados
PRECIOS, RUTA = Precios(TARIFAS)
#print(PRECIOS)
#print(RUTA)

def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    return ''#antes desde
  
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta]))+'-'+str(RUTA[desde][hasta])

#Probamos el algoritmo para ir de 0 a 6:
print('Ruta óptima para ir del nodo (0) al nodo (6): '+calcular_ruta(RUTA,0,6))
print('')
print('')

Ruta óptima para ir del nodo (0) al nodo (6): -0-2-5




## Actividad Guiada 2 - Ramificación y Poda

In [20]:
#Librerías para cálculo de tiempos de ejecución
import itertools
from functools import wraps

#función decorador para calcular tiempos
def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()
        salida = f(*args, **kwargs)
        t_final = time()
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

In [21]:
#Caso de asignación de tareas a N agentes
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [22]:
def valor(S, COSTES):
    VALOR = 0
    #Valores establecidos
    for i in range(len(S)):
        VALOR += COSTES[i][S[i]]
      
    return VALOR

#Probamos algoritmo
valor((0,1,2,3),COSTES)

73

In [26]:
def CI(S,COSTES):
    VALOR = 0
    
    for i in range(len(S)):
        VALOR += COSTES[i][S[i]]
    
     #Estimación
    for i in range(len(COSTES)):
        if i not in S:
            VALOR += min([ COSTES[j][i] for j in range(len(S), len(COSTES)) ])
                          
    return VALOR

CI((1,2),COSTES)

59

In [28]:
def crear_hijos(NODO,N):
    HIJOS = []
    for i in range(N):
        if i not in NODO:
            HIJOS.append( {'s':NODO+(i,)} )
    return HIJOS

crear_hijos((0,2),4)

[{'s': (0, 2, 1)}, {'s': (0, 2, 3)}]

In [35]:
def ramificacion_y_poda(COSTES):
    DIMENSION = len(COSTES)
    MEJOR_SOLUCION = tuple(i for i in range(DIMENSION))
    CotaSup = valor(MEJOR_SOLUCION,COSTES)
    
    NODOS = []
    NODOS.append({'s':(),'ci':CI((),COSTES)})
    
    print(NODOS)
    iteracion = 0
    
    while (len(NODOS)>0):
        iteracion+=1
        print('\n#',iteracion)
        
        nodo_prometedor = min(NODOS, key=lambda x:x['ci'])
        
        #hijos
        HIJOS = [{'s':x['s'],'ci':CI(x['s'],COSTES)} for x in crear_hijos(nodo_prometedor['s'],DIMENSION)]
        
        NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
        if len(NODO_FINAL) > 0:
            if NODO_FINAL[0]['ci'] < CotaSup:
                CotaSup = NODO_FINAL[0]['ci']
                MEJOR_SOLUCION = NODO_FINAL[0]
                        
        #Poda
        HIJOS = [ x for x in HIJOS if x['ci']<CotaSup ]
        print(nodo_prometedor)
        #Eliminamos el nodo expandido
        NODOS = [ x for x in NODOS if x['s']!=nodo_prometedor['s'] ]
        
        #Añadimos los hijos
        NODOS.extend(HIJOS)
        
        print("la mejor solucion es: ",MEJOR_SOLUCION)
    
ramificacion_y_poda(COSTES)

[{'s': (), 'ci': 58}]

# 1
{'s': (), 'ci': 58}
la mejor solucion es:  (0, 1, 2, 3)

# 2
{'s': (1,), 'ci': 58}
la mejor solucion es:  (0, 1, 2, 3)

# 3
{'s': (1, 2), 'ci': 59}
la mejor solucion es:  (0, 1, 2, 3)

# 4
{'s': (0,), 'ci': 60}
la mejor solucion es:  (0, 1, 2, 3)

# 5
{'s': (0, 2), 'ci': 61}
la mejor solucion es:  (0, 1, 2, 3)

# 6
{'s': (0, 2, 3), 'ci': 61}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci': 61}

# 7
{'s': (1, 3), 'ci': 64}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci': 61}

# 8
{'s': (1, 2, 0), 'ci': 64}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci': 61}

# 9
{'s': (2,), 'ci': 65}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci': 61}

# 10
{'s': (1, 2, 3), 'ci': 65}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci': 61}

# 11
{'s': (0, 3), 'ci': 66}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci': 61}

# 12
{'s': (1, 0), 'ci': 68}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci': 61}

# 13
{'s': (0, 1), 'ci': 68}
la mejor solucion es:  {'s': (0, 2, 3, 1), 'ci'